In [49]:
# Importing necessary modules

import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil



# Identifying directory of lab sheets for quality screening

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)



# Setting loop for iterating through files WITHIN folders IN the specified folder

for x, folder, file in os.walk(datasheets):
    
    
    # Using "try" loop in the event that a folder is empty
    
    try: 
        
        
        # Looping through every file in a folder
        
        for loc in folder:
            
            
            # Identifying working folder
            
            print(loc)

            folder_loc = datasheets + loc

            glob.os.chdir(folder_loc)
            
            
            # Grabbing all csv files in specified folder

            all_datasheets = glob.glob(folder_loc + "/*.csv")
            
            
            # Creating empty lists to append with ID's that fail quality screening

            N_Re_Run = []
            U_Re_Run = []
            
            
            # Screening each sheet individually

            for sheet in all_datasheets:

                
                # Reading working file as dataframe in pandas
                
                lab_data = pd.read_csv(sheet)
                
                
                # Pulling parts of name to identify sheet for naming/moving

                sheet_name = sheet.split('\\')[-1]
                title = sheet_name.split('_')[2]
            
            
                # Checking for data - skipping remainder of code (breaking loop) if there is no data

                if np.isnan(lab_data.Absorbance[0]):
                    break

                
                # Continuing if data is found

                else:
                    
                    # Identifying sheet
                    
                    print(sheet)


                    # Creating empty lists to append with curve information
                    
                    curve_absorbance = []
                    curve_concentration = []

                    
                    # Identifying standard absorbances based on 'Type' column
                    
                    for n in range(len(lab_data)):
                        if lab_data.Type[n] == 'C':
                            
                            
                            # Appending calibration data to appropriate list for curve analysis
                            
                            curve_absorbance.append(float(lab_data.Absorbance[n]))
                            curve_concentration.append(float(lab_data.Sample_ID[n]))


                # Calculating r^2 value for two calibration data lists with lin.regress module

                slope, intercept, r_value, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)

                print(r_value**2)
                
                # Creating conditional for screening based on r^2 value
                
                if r_value**2 < 0.95:
                    
                    columns = [['Conc', 'Abs']]
                    
                    curve = pd.DataFrame(columns = columns)

                    curve['Conc'] = curve_concentration
                    curve['Abs'] = curve_absorbance

                    points = len(curve_concentration) # Total number

                    array = np.identity(points)

                    for i in range(array.shape[1]):

                        curve[str(i)] = array[:,i]

                    for i in range(1,points):

                        array = np.identity(i)
                        array = np.vstack([array, [1]*i])

                        if array.shape[0] != 12:
                            for j in range(12 - array.shape[0]):
                                array = np.vstack([array, [0]*array.shape[1]])

                            for k in range(array.shape[1]):

                                curve[str(i)+str(k)+str(k)] = array[:,k]
                    columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                    columns = []
                    slopes = []
                    intercepts = []
                    rvals = []
                    pvals = []
                    stderrs = []

                    parameters = pd.DataFrame(columns = columns)

                    for column in curve.iloc[:, 2:14]:

                        columns.append(column)

                        count = 0
                        conc = []
                        absorb = []


                        for i in curve[column]: 
                            if i < 1:
                                conc.append(curve.iloc[count,0])
                                absorb.append(curve.iloc[count,1])
                            count = count + 1

                        slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                        slopes.append(slope)
                        intercepts.append(inter)
                        rvals.append(r_val**2)
                        pvals.append(p_val)
                        stderrs.append(std)

                    parameters['Column'] = columns
                    parameters['Slope'] = slopes
                    parameters['Intercept'] = intercepts
                    parameters['R_Sq'] = rvals
                    parameters['P_Val'] = pvals
                    parameters['StdErr'] = stderrs

                    print(parameters.R_Sq.max())
                    print(parameters)

                    if parameters['R_Sq'].max() < 0.95:

                        columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                        columns = []
                        slopes = []
                        intercepts = []
                        rvals = []
                        pvals = []
                        stderrs = []

                        parameters = pd.DataFrame(columns = columns)

                        for column in curve.iloc[:, 14:]:

                            columns.append(column)

                            count = 0
                            conc = []
                            absorb = []


                            for i in curve[column]: 
                                if i < 1:
                                    conc.append(curve.iloc[count,0])
                                    absorb.append(curve.iloc[count,1])
                                count = count + 1

                            slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                            slopes.append(slope)
                            intercepts.append(inter)
                            rvals.append(r_val**2)
                            pvals.append(p_val)
                            stderrs.append(std)
                        #     print(conc)
                        #     print(r_val**2)
                        parameters['Column'] = columns
                        parameters['Slope'] = slopes
                        parameters['Intercept'] = intercepts
                        parameters['R_Sq'] = rvals
                        parameters['P_Val'] = pvals
                        parameters['StdErr'] = stderrs

                        print(parameters.R_Sq.max())
                    
                    if parameters['R_Sq'].max() >= 0.95:
                        
                        print("Good curve")
                        shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Lab_Sheets/' + loc + '/Completed/' + str(sheet_name))
                    else:
                        # Marking bad curve sheet and moving to 'Completed' folder to avoid re-running in the future

                        print('Bad curve')
                        shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Lab_Sheets/' + loc + '/Completed/' + 'x_BAD_CURVE_' +str(sheet_name))


                        # Adding Sample ID information from bad calibration to appropriate list of re-runs

    except:
        continue

MR_USB
RV_CTS
USB_Indiana
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_Indiana\A_5_URE_LMR_IN_USB_03_21_19.csv
0.5441681647508237
0.9999138536496541
   Column     Slope  Intercept      R_Sq         P_Val    StdErr
0    (0,)  0.000353   0.120326  0.519684  1.230824e-02  0.000113
1    (1,)  0.000353   0.120326  0.519684  1.230824e-02  0.000113
2    (2,)  0.000353   0.120500  0.519628  1.231522e-02  0.000113
3    (3,)  0.000358   0.101110  0.537632  1.024081e-02  0.000111
4    (4,)  0.000358   0.101950  0.537364  1.026943e-02  0.000111
5    (5,)  0.000358   0.101950  0.537364  1.026943e-02  0.000111
6    (6,)  0.000359   0.092442  0.545314  9.447433e-03  0.000109
7    (7,)  0.000359   0.089116  0.546341  9.345245e-03  0.000109
8    (8,)  0.000359   0.089892  0.546090  9.370168e-03  0.000109
9    (9,)  0.000268   0.148162  0.387503  4.081082e-02  0.000112
10  (10,)  0.000551   0.007226  0.999914  1.322394e-19  0.000002
11  (11,)  0.000266   0.149185  0.386900  4.101407e-02  0

In [36]:
columns = ['Conc', 'Abs']

curve = pd.DataFrame(columns = columns)

curve['Conc'] = curve_concentration
curve['Abs'] = curve_absorbance

points = len(curve_concentration) # Total number

array = np.identity(points)

for i in range(array.shape[1]):

    curve[str(i)] = array[:,i]

for i in range(1,points):

    array = np.identity(i)
    array = np.vstack([array, [1]*i])

    if array.shape[0] != 12:
        for j in range(12 - array.shape[0]):
            array = np.vstack([array, [0]*array.shape[1]])

        for k in range(array.shape[1]):

            curve[str(i)+str(k)+str(k)] = array[:,k]
columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

columns = []
slopes = []
intercepts = []
rvals = []
pvals = []
stderrs = []

parameters = pd.DataFrame(columns = columns)

for column in curve.iloc[:, 2:14]:
    
    columns.append(column)
    
    count = 0
    conc = []
    absorb = []
    
    
    for i in curve[column]: 
        if i < 1:
            conc.append(curve.iloc[count,0])
            absorb.append(curve.iloc[count,1])
        count = count + 1
        
    slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)
    
    slopes.append(slope)
    intercepts.append(inter)
    rvals.append(r_val**2)
    pvals.append(p_val)
    stderrs.append(std)

parameters['Column'] = columns
parameters['Slope'] = slopes
parameters['Intercept'] = intercepts
parameters['R_Sq'] = rvals
parameters['P_Val'] = pvals
parameters['StdErr'] = stderrs

print(parameters.R_Sq.max())

if parameters['R_Sq'].max() < 0.90:
    
    columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

    columns = []
    slopes = []
    intercepts = []
    rvals = []
    pvals = []
    stderrs = []

    parameters = pd.DataFrame(columns = columns)

    for column in curve.iloc[:, 14:]:

        columns.append(column)
        
        count = 0
        conc = []
        absorb = []


        for i in curve[column]: 
            if i < 1:
                conc.append(curve.iloc[count,0])
                absorb.append(curve.iloc[count,1])
            count = count + 1

        slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

        slopes.append(slope)
        intercepts.append(inter)
        rvals.append(r_val**2)
        pvals.append(p_val)
        stderrs.append(std)
    #     print(conc)
    #     print(r_val**2)
    parameters['Column'] = columns
    parameters['Slope'] = slopes
    parameters['Intercept'] = intercepts
    parameters['R_Sq'] = rvals
    parameters['P_Val'] = pvals
    parameters['StdErr'] = stderrs
    
parameters

0.8587038805948787


,Column,Slope,Intercept,R_Sq,P_Val,StdErr
0,100,0.000354,-0.245000,0.581206,0.010353,0.000106
1,200,0.000342,-0.201250,0.556845,0.013184,0.000108
2,211,0.000354,-0.245000,0.581206,0.010353,0.000106
3,300,0.000319,-0.110909,0.578776,0.010612,0.000096
4,311,0.000328,-0.144793,0.597123,0.008779,0.000095
5,322,0.000319,-0.110909,0.578776,0.010612,0.000096
6,400,0.000320,-0.114876,0.579201,0.010566,0.000096
7,411,0.000329,-0.148760,0.597592,0.008736,0.000095
8,422,0.000320,-0.114876,0.579201,0.010566,0.000096
9,433,0.000306,-0.055556,0.604321,0.008133,0.000088
